In [282]:
import pandas as pd
import os

In [283]:
excel = os.listdir(r'dados/')

In [284]:
arquivos_unicificados = []
for i, arquivo in enumerate(excel):
    df = pd.read_excel(f'dados/{arquivo}')
    df.loc[:, 'ciclo'] = f'ciclo {i+1}'
    arquivos_unicificados.append(df)

arquivos_unicificados = pd.concat(arquivos_unicificados)

In [ ]:
arquivos_unicificados = arquivos_unicificados.rename(columns={'Veículo':'veiculo', 'Lote':'lote', 'Empresa':'empresa'})
arquivos_unicificados.loc[: ,'data'] = arquivos_unicificados.dt_levantamento.dt.date
arquivos_unicificados.loc[:, 'horario'] = arquivos_unicificados.dt_levantamento.dt.time.astype(object)
arquivos_unicificados = arquivos_unicificados.drop('dt_levantamento', axis=1)
arquivos_unicificados = arquivos_unicificados[['veiculo','data', 'horario','uf','trecho','km','sentido','lote','ciclo']]

In [286]:
lista_dfs={}
veiculos = arquivos_unicificados.veiculo.unique()
sentidos = arquivos_unicificados.sentido.unique()

for veiculo in veiculos:
    dfs_filtrados = []
    for sentido in sentidos:
        subset = arquivos_unicificados.loc[(arquivos_unicificados.veiculo == veiculo) 
                                            & (arquivos_unicificados.sentido == sentido)].copy()
        if subset.empty:
            pass
        else:
            subset = subset.sort_values(by=['ciclo','data','horario','uf','trecho','km']).reset_index(drop=True)
            dfs_filtrados.append(subset)
    if len(dfs_filtrados) == 0:
        pass
    else:
        lista_dfs[veiculo] = dfs_filtrados

In [287]:
def dist_percorrida (df):
    df['distancia_percorrida'] = pd.Series(dtype='float64')
    df['motivo'] = pd.Series(dtype='object')
    df_index = df.index

    for i in df_index:
        if i == 0:
            df.loc[i,'distancia_percorrida'] = 0
            df.loc[i,'motivo'] = 'primeira linha'
        elif df.loc[i, 'data'] != df.loc[i-1, 'data']:
            df.loc[i,'distancia_percorrida'] = 0
            df.loc[i,'motivo'] = 'inicio do dia'
        elif df.loc[i, 'km'] <= df.loc[i-1, 'km']:
            df.loc[i, 'distancia_percorrida'] = 0
            df.loc[i,'motivo'] = 'km menor/igual ao anterior'
        else:
            df.loc[i, 'distancia_percorrida'] = df.loc[i, 'km'] - df.loc[i-1, 'km']

    threshold = 2 * df['distancia_percorrida'].std()
    for i in df_index:
        if df.loc[i, 'distancia_percorrida'] >=  threshold:
            df.loc[i, 'distancia_percorrida'] = 0
            df.loc[i,'motivo'] = 'km maior que 2*desv.pad'

    return  df

In [288]:
b = []

for veiculos in lista_dfs.values():
    for df in veiculos:
        dist_percorrida(df)
        b.append(df)

In [290]:
c = pd.concat(b, ignore_index=True)

pd.DataFrame(c.groupby(['ciclo', 'lote', 'uf', 'veiculo', 'data'])['distancia_percorrida'].sum() /1000).to_csv('teste.csv', decimal=',')


In [291]:
pd.concat(b).to_csv('b.csv', decimal=',')